# Basic SQL

1.
Count the number of companies that have closed.

In [ ]:
SELECT COUNT(status)
FROM company
WHERE status = 'closed'

2.
Use the company table data to display the amount of funds raised for US news companies. Sort the table in descending order by funding_total.

In [ ]:
SELECT SUM(funding_total)
FROM company
WHERE category_code = 'news'
  AND country_code = 'USA'
GROUP BY name, 
         funding_total
ORDER BY funding_total desc

3.
Find the dollar amount of all transactions for some companies purchased by others from 2011 through 2013. Select cash-only transactions. 

In [ ]:
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash'
  AND CAST(acquired_at AS date) BETWEEN '2011-01-01' AND '2013-12-31'

4.
Display the first name, last name, and twitter account name of people whose account names begin with 'Silver'.

In [ ]:
SELECT first_name,
       last_name,
       twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%'

5.
Display the information about people whose twitter account names contain the substring 'money' and whose last name begins with 'K'.

In [ ]:
SELECT *
FROM people
WHERE twitter_username LIKE '%money%'
  AND last_name LIKE 'K%'

6.
In each country, show the total amount of investments received by companies registered in the country. The country code of the company can be used to identify the country in which the company is registered. Sort in descending order.

In [ ]:
SELECT country_code,
       SUM(funding_total)
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC

7.
Create a table that includes the date of the round, as well as the minimum and maximum amount of investments attracted on this date. Only include records in which the minimum value does not equal zero or the maximum value. 

In [ ]:
SELECT CAST(funded_at AS date),
       MIN(raised_amount),
       MAX(raised_amount)
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) != 0
   AND MIN(raised_amount) != MAX(raised_amount)

8.
Create a field with categories:
   - Assign the high_activity category to funds that invest in 100 or more companies;
   - Assign the middle_activity category to funds that invest in 20 or more companies up to 100;
   - If the number of invested fund companies does not reach 20, assign the low_activity category.

Display all fields of the fund table and a new category field.

In [ ]:
SELECT *,
       CASE 
           WHEN invested_companies >= 100 THEN 'high_activity'
           WHEN invested_companies >=20 AND invested_companies < 100 THEN 'middle_activity'
           WHEN invested_companies < 20 THEN 'low_activity'
       END
FROM fund

9.
Calculate the average number of investment rounds the fund has participated in for each category assigned in the previous task. Round the numbers to the nearest whole number. Display the categories and the average number of investment rounds. Sort the table in ascending order.

In [ ]:
SELECT
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
       ROUND(AVG(investment_rounds), 0)
FROM fund
GROUP BY activity
ORDER BY AVG(investment_rounds)

10.
Download the table with the top ten active investing countries. Determine the country's activity based on the average number of companies in which its funds invest.

In each country calculate the minimum, maximum, and average number of companies invested in by funds founded between 2010 and 2012.

Exclude countries where funds with minimum number of companies that have received investments equal to zero. 

Sort the table by the average number of companies in descending order, and then by country code in lexicographic order.

Use the BETWEEN operator to filter a range by years.

In [ ]:
SELECT country_code,
       AVG(invested_companies), 
       MIN(invested_companies),
       MAX(invested_companies)
FROM fund
WHERE EXTRACT(year FROM CAST(founded_at AS date)) BETWEEN '2010' AND '2012'
GROUP BY country_code
HAVING MIN(invested_companies) != 0
ORDER BY AVG(invested_companies) DESC
LIMIT 10

11.
Display the first and last name of all startup employees. If this information is available, also include the name of the educational institution that employee graduated from, .

In [ ]:
SELECT p.first_name,
       p.last_name,
       e.instituition
FROM people AS p
LEFT OUTER JOIN education e ON p.id=e.person_id

12.
Identify the number of educational institutions from which each copmany's employees graduated. Print the name of the company and the number of unique school names. Rank the top 5 companies by the number of universities.

In [ ]:
SELECT c.name,
       COUNT(DISTINCT e.instituition)
FROM company AS c
INNER JOIN people p ON c.id = p.company_id
INNER JOIN education e ON p.id = e.person_id
GROUP BY c.name
ORDER BY count DESC
LIMIT 5

13.
Make a list with unique closed companies whose first funding round was the last one.

In [ ]:
SELECT DISTINCT name
FROM company 
WHERE status = 'closed'
  AND id IN (SELECT company_id
             FROM funding_round
             WHERE is_last_round =1
               AND is_first_round =1)

14.
Make a list of unique employee ids for the companies selected in the previous step.

In [ ]:
SELECT DISTINCT p.id
FROM people AS p
WHERE p.company_id IN (SELECT c.id
                       FROM company c
                       WHERE c.status = 'closed'
                         AND c.id IN (SELECT fr.company_id
                                      FROM funding_round as fr
                                      WHERE fr.is_last_round =1
                                        AND fr.is_first_round =1))

15.
Create a table that includes unique pairs of employee ids from the previous step and the educational institution from which the employee graduated.

In [ ]:
SELECT DISTINCT e.person_id,
                e.instituition
FROM education e
WHERE e.person_id IN (SELECT p.id
                      FROM people p
                      WHERE p.company_id IN (SELECT c.id
                                             FROM company c
                                             WHERE c.status = 'closed'
                                             AND c.id IN (SELECT fr.company_id
                                                          FROM funding_round fr
                                                          WHERE fr.is_last_round =1
                                                          AND fr.is_first_round =1)))

16.
Calculate the number of educational institutions for each employee from the previous task.

In [ ]:
SELECT DISTINCT e.person_id,
       COUNT(e.instituition)
FROM education e
WHERE e.person_id IN (SELECT p.id
                      FROM people p
                      WHERE p.company_id IN (SELECT c.id
                                             FROM company c
                                             WHERE c.status = 'closed'
                                               AND c.id IN (SELECT fr.company_id
                                                            FROM funding_round AS fr
                                                            WHERE fr.is_last_round =1
                                                              AND fr.is_first_round =1)))
GROUP BY e.person_id

17.
Complete the previous query and print the average number of educational institutions that employees of different companies graduated from (all, not just unique ones). Displaying only one record is sufficient, grouping is not required. 

In [ ]:
SELECT SUM(count)/COUNT(*)
FROM 
     (SELECT DISTINCT e.person_id,
             COUNT(e.instituition)
      FROM education e
      WHERE e.person_id IN (SELECT p.id
                            FROM people p
                            WHERE p.company_id IN (SELECT c.id
                                                   FROM company c
                                                   WHERE c.status = 'closed'
                                                     AND c.id IN (SELECT fr.company_id
                                                                  FROM funding_round fr
                                                                  WHERE fr.is_last_round =1
                                                                    AND fr.is_first_round =1)))
      GROUP BY e.person_id) AS a

18.
Write a similar query: print the average number of schools (all, not just the unique ones) Facebook employees have graduated from. 

In [ ]:
SELECT SUM(count)/COUNT(*)
FROM (SELECT DISTINCT e.person_id,
             COUNT(e.instituition)
      FROM education AS e
      WHERE e.person_id IN (SELECT p.id
                            FROM people AS p
                            WHERE p.company_id IN (SELECT c.id
                                                   FROM company c
                                                   WHERE c.name = 'Facebook'))
                                                   GROUP BY e.person_id) AS a

19.
Create a table:
- name_of_fund — name of the fund;
- name_of_company — company name;
- amount — the amount of investments that the company raised in the round.

The table will include data on companies whose history includes more than six significant milestones, and whose funding rounds occured between 2012 to 2013.

In [ ]:
SELECT f.name AS name_of_fund,
       c.name AS name_of_company,
       fr.raised_amount AS amount
FROM investment AS i
LEFT JOIN company c ON i.company_id = c.id
LEFT JOIN fund f ON i.fund_id = f.id
LEFT JOIN funding_round fr ON i.funding_round_id = fr.id
WHERE c.milestones > 6
  AND EXTRACT(year from CAST(fr.funded_at AS date)) BETWEEN '2012' AND '2013'

20.
Download a table that contains the following fields:
- company-buyer's name;
- transaction amount;
- the name of the company that was acquired;
- investments made in the company acquired;
- the proportion of the purchase price over the amount invested in the company, rounded up to the nearest whole number.

The table should exclude transactions in which the purchase amount is zero. If the amount of investment in a company is zero, exclude it from the table.

The table should be sorted by transaction amount from largest to smallest, and then by company name in lexicographic order. Limit the table to the first ten records.

In [ ]:
SELECT c.name AS acquiring_company_name,
       a.price_amount AS deal_amount,
       co.name AS acquired_company_name,
       co.funding_total AS investment_amount, 
       ROUND(a.price_amount/co.funding_total) AS rate
FROM acquisition a
LEFT JOIN company c ON a.acquiring_company_id = c.id
LEFT JOIN company co ON a.acquired_company_id = co.id
WHERE a.price_amount != 0 
  AND co.funding_total != 0
ORDER BY deal_amount DESC, 
         acquired_company_name
LIMIT 10

21.
Download the table, which includes the names of companies from category social that received limited funding from 2010 to 2013. You should include the month in which the funding round took place.

In [ ]:
SELECT c.name,
       EXTRACT(month FROM cast(fr.funded_at AS date))
FROM company c
LEFT JOIN funding_round fr ON c.id = fr.company_id
WHERE c.category_code = 'social'
AND EXTRACT(year FROM CAST(fr.funded_at AS date)) BETWEEN '2010' AND '2013'

22.
Select data by month between 2010 and 2013 when investment rounds were held. Group the data by month number and get a table with following fields:
- the month number when the investment rounds were held;
- the amount of unique US funds that have invested this month;
- the number of companies purchased this month;
- the total amount of purchases this month.

In [ ]:
WITH
a AS (SELECT EXTRACT(month 
      FROM CAST(funded_at AS date)) AS month,
      COUNT(DISTINCT f.id) AS count_fund
      FROM funding_round AS fr
      LEFT JOIN investment i ON fr.id = i.funding_round_id
      LEFT JOIN fund f ON i.fund_id = f.id
      WHERE EXTRACT(year FROM CAST(funded_at AS date)) BETWEEN '2010' AND '2013'
        AND f.country_code = 'USA'
      GROUP BY month
     ),

b AS (SELECT extract(month 
      FROM CAST(acquired_at AS date)) AS month,
      COUNT(acquired_company_id) AS count_acquired_company,
      SUM(price_amount) AS sum_price_amoun
      FROM acquisition
      WHERE EXTRACT(year FROM CAST(acquired_at AS date)) BETWEEN '2010' AND '2013'
      GROUP BY month
     )
SELECT a.month,
       a.count_fund,
       b.count_acquired_company,
       b.sum_price_amoun
FROM a
LEFT JOIN b ON a.month = b.month
ORDER BY month

23.
Create a pivot table and calculate the average investment amount for countries with startups registered in 2011, 2012 and 2013. Each year's data should be in a separate field. Sort the table by the average investment amount for 2011 from highest to lowest.

In [ ]:
WITH
one AS (SELECT AVG(c.funding_total) AS finance1,
               c.country_code
        FROM company AS c
        WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) ='2011' 
        GROUP BY c.country_code),
two AS (SELECT AVG(c.funding_total) AS finance2,
               c.country_code
        FROM company AS c
        WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) ='2012'
        GROUP BY c.country_code),
three AS (SELECT AVG(c.funding_total) AS finance3,
                 c.country_code
          FROM company AS c
          WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) ='2013'
          GROUP BY c.country_code)
 
SELECT one.country_code,
       one.finance1 AS invested_2011,
       two.finance2 AS invested_2012,
       three.finance3 AS invested_2013
FROM one
INNER JOIN two ON two.country_code=one.country_code
INNER JOIN three ON three.country_code=two.country_code
ORDER BY invested_2011 DESC